In [1]:
from google.cloud import bigquery
import pandas as pd

In [3]:
# credpath ='C:/Users/jakes/OneDrive/Documents/Python/credentials.json'
# credentials = credentials.Credentials.from_authorized_user_file(credpath)
# client = bigquery.Client(credentials=credentials)
client = bigquery.Client()

zonequery = """
SELECT TT.dma_name, term, SUM(score) as TOTAL_SCORE, RANK() OVER (PARTITION BY TT.dma_name ORDER BY SUM(score) DESC) AS TERM_RANK
FROM `bigquery-public-data.google_trends.top_terms` TT
WHERE
    1=1
   AND refresh_date = (SELECT MAX(refresh_date) FROM `bigquery-public-data.google_trends.top_terms`)
GROUP BY 1,2
ORDER BY 1 ASC,4 ASC;
"""
df = pd.read_gbq(zonequery)
totalquery = """
SELECT term, SUM(score) as TOTAL_SCORE, RANK() OVER (ORDER BY SUM(score) DESC) AS TERM_RANK
FROM `bigquery-public-data.google_trends.top_terms` TT
WHERE
    1=1
   AND refresh_date = (SELECT MAX(refresh_date) FROM `bigquery-public-data.google_trends.top_terms`)
GROUP BY 1
ORDER BY 3 ASC;
"""
appenddf = pd.read_gbq(totalquery)
appenddf['dma_name'] = 'TOTAL'
df = pd.concat([df,appenddf])
df.to_csv('C:/Users/jakes/Documents/TopGoogleSearchTerms.csv')